In [12]:
import copy
from collections import deque


# AC3 heuristic - checks whether it is arc consistent or not
def AC3_inference(deque_array, adjancy_list, domains_new):

    while len(deque_array) != 0 :
        
        new_arr = deque_array.popleft()
        #print("new arr", new_arr)
        
        [remove,domains_new] = revise(new_arr, domains_new)

        if remove == 1:
            if len(domains_new[new_arr[0]])==0:
                return (0,domains_new)
            
            # if the result coming from revise is True, then it takes v2 from [v1,v2] and append the neighbors
            #    of v2 to deque.
            adj_lists = [item for item in adjancy_list[new_arr[0]] if item != new_arr[1]]
            
            for adj_list in adj_lists:
                if colors[adj_list] == -1:
                    deque_array.append([adj_list, new_arr[0]])

    return (1, domains_new)



def revise(new_arr,domains_new):

    # if inconsistent values removed from domains_new list, it gives back True and new domains_list, otherwise False.
    
    revised = 0
    #print("domains_new", domains_new[new_arr[1]])
    
    if len(domains_new[new_arr[1]]) == 1 :
        
        check_color = domains_new[new_arr[1]][0]
        
        if check_color in domains_new[new_arr[0]] :
            
            domains_new[new_arr[0]].remove(check_color)
            revised = 1
            
    return (revised,domains_new)


# LCV heuristic
def order_colors(vertex, adjancy_list, domains) :
  #print("Order: ", end=" ")
    order = []
    
    for color in domains[vertex] :

        constraint_count = 0
        
        for adj_vertex in adjancy_list[vertex] :

            if color in domains[adj_vertex] :
                constraint_count += 1
        
        order.append([color,constraint_count])

        #print(order)
        #
    colors = [item[0] for item in sorted(order, key = lambda val: val[1])]
    #print(colors)
    return colors



# MRV heuristic
def select_next_vertex(Vertex_count, domains, colors): 
    
    mini = 1000
    nexti = -1

    for vertex in range(Vertex_count):
        
        if ( len(domains[vertex]) < mini and colors[vertex] == -1) :
            
            mini = len(domains[vertex])
            #print("->", mini,end=" ")
            nexti = vertex

    #print(" nexti ->", nexti)
    return nexti


def is_safe(next_vertex, color, colors, adjancy_list) :
    for adj_vertex in adjancy_list[next_vertex]:
        #print(adj_vertex)
        if color == colors[adj_vertex]:
            return 0

    return 1



def backtracking_search(Vertex_count, colors, adjancy_list, domains):
    return backtrack(Vertex_count, colors, adjancy_list, domains)

def backtrack(Vertex_count, colors, adjancy_list, domains):
    
    # to prevent maximum recursion depth exceeded error, if all vertices colored, stops backtracking
    if -1 not in colors:
        return colors

    # select next vertex using MRV heuristic
    next_vertex = select_next_vertex(Vertex_count, domains, colors)
    #print(next_vertex,"->")
    
    # select color order using LCV heuristic
    color_order = order_colors(next_vertex, adjancy_list, domains)
    #print(color_order)
    
    
    #color_order = domains[next_vertex]
    
    
    for color in color_order:

        domains_new = copy.deepcopy(domains)
        #print(domains_new)
        
        if (is_safe(next_vertex, color, colors, adjancy_list)):
            
            domains_new[next_vertex] = [item for item in domains[next_vertex] if item == color]
            #print(domains_new)
            
            
            array_unassigned_adj_vertices = []
            
            
            for adj_vertex in adjancy_list[next_vertex]:
                if colors[adj_vertex]== -1:
                    array_unassigned_adj_vertices.append([adj_vertex,next_vertex])
            #print(array)
            
            
            deque_array = deque(array_unassigned_adj_vertices)
            #print(deque_array)
            
            
            [consistent, domain_last] = AC3_inference(deque_array, adjancy_list, domains_new)
    
            if consistent == 1 :
            
                colors[next_vertex] = color
                
                domains_new = domain_last
                
                result = backtrack(Vertex_count, colors, adjancy_list, domains_new)
                
                if result != 0 :
                    return result
        
        colors[next_vertex] = -1
        
    return 0




def checktruth(Vertex_count, adjancy_list, result) :
    for vertex in range(Vertex_count) :
        for adj_vertex in adjancy_list[vertex] :
            if result[vertex] == result[adj_vertex] :
                return "Match!"
    return "No match! Done!"    
    

if __name__ == "__main__" :
    
    # Input file name
    file_name = input("Enter file name : ")
    #"input4.txt"
    #input("Enter file name : ")
    
    vertices = []
    
    # to set limit to the input file
    counter = 0
    
    # reading input file
    with open(file_name, 'r') as reader :
        for line in reader :
            if line[0] != '#' :
                if line[0] == 'c' :
                    # taking color_size
                    color_size = int(line.split("= ")[1])
                else :          
                    # add vertices to the list
                    vertices.append(int(line.split(",")[0]))
                    vertices.append(int(line.split(",")[1]))
                    
                    counter += 1
                    #print(counter)
                    
            if counter == 1000 :
                break
                
    #print(vertices)
    
    Vertex_count = max(vertices) + 1
    #print(Vertex_count)
    

    adjancy_list = [ [] for vertex in range(Vertex_count) ]
    #print(adjancy_list)

    # create adjancy list
    for i in range(0, len(vertices), 2) : 
        #print(i," ", vertices[i], " ", vertices[i+1])
        
        # it is undirected graph
        adjancy_list[vertices[i]].append(vertices[i + 1])
        adjancy_list[vertices[i + 1]].append(vertices[i])
        
    #print(adjancy_list)
    
      
    domains = [ [] for vertices in range(Vertex_count)]
    #print(domains)
    
    for vertex in range(0, Vertex_count) :
        for color in range(0, color_size) :
            domains[vertex].append(color)

    #print(domains)

    
    # define colors for the vertices
    colors = [ -1 for vertex in range(Vertex_count)]
    #print(colors)
    
    # call backtracking search
    result = backtracking_search(Vertex_count, colors, adjancy_list, domains)
    #print(result)    
    
    color_names = ['blue','red','green','black','white','yellow','silver','gold','brown','purple','pink','orange','gray',
                'aliceblue','antiquewhite','aqua','aquamarine','azure','beige','bisque','blanchedalmond',
             'blueviolet','burlywood','cadetblue','chartreuse','chocolate','coral','cornflowerblue',
              'cornsilk','crimson','cyan','darkblue','darkcyan','darkgoldenrod','darkgray','darkgreen','darkkhaki',
                'darkmagenta','darkolivegreen','darkorange','darkorchid','darkred','darksalmon','darkseagreen','darkslateblue',
                'darkslategray','darkturquoise','darkviolet','deepskyblue','dimgray','dodgerblue','firebrick',
                'floralwhite','forestgreen','fuchsia','gainsboro','ghostwhite','goldenrod',
                'greenyellow','honeydew','hotpink','indianred','indigo','ivory','khaki','lavender','lavenderblush',
                'lawngreen','lemonchiffon','lightblue','lightcoral','lightcyan','lightgoldenrodyellow','lightgreen',
                'lightgray','lightpink','lightsalmon','lightseagreen','lightskyblue','lightslategray','lightsteelblue',
                'lightyellow','lime','limegreen','linen','magenta','maroon','mediumaquamarine','mediumblue','mediumorchid',
                'mediumpurple','mediumseagreen','mediumslateblue','mediumspringgreen','mediumturquoise','mediumvioletred',                'midnightblue','mintcream',
                'mistyrose','moccasin','navajowhite','navy','oldlace','olive','olivedrab','orangered','orchid',
                'palegoldenrod','palegreen','paleturquoise','palevioletred','papayawhip','peachpuff','peru','plum',
                'powderblue','rosybrown','royalblue','saddlebrown','salmon','sandybrown','seagreen','seashell',
                'sienna','skyblue','slateblue','slategray','snow','springgreen','steelblue','tan','teal','thistle',
                'tomato','turquoise','violet','wheat','whitesmoke','yellowgr']
    
    if result == 0 :
        print("\nIt is not possible!")
    else:
        print("\nIt is possible!\n")
        for vertex in range(len(result)) :
            print(vertex, "'th vertex -> ", color_names[result[vertex]])
        print("\n",checktruth(Vertex_count, adjancy_list, result))

Enter file name : input4.txt

It is possible!

0 'th vertex ->  blue
1 'th vertex ->  red
2 'th vertex ->  red
3 'th vertex ->  green
4 'th vertex ->  green
5 'th vertex ->  black
6 'th vertex ->  black
7 'th vertex ->  blue
8 'th vertex ->  red
9 'th vertex ->  red
10 'th vertex ->  green
11 'th vertex ->  black
12 'th vertex ->  green
13 'th vertex ->  red
14 'th vertex ->  blue
15 'th vertex ->  green
16 'th vertex ->  black

 No match! Done!
